In [1]:
@file:DependsOn("io.ktor:ktor-client-core:3.1.3")
@file:DependsOn("io.ktor:ktor-client-cio:3.1.3")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:3.1.3")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:3.1.3")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe

import ro.jf.funds.client.notebook.*
import kotlinx.coroutines.runBlocking

val client = FundsClient()
val user = runBlocking { client.ensureUserExists("Johann-20.3") }
user

UserTO(id=e972a763-dd79-40e9-8ba9-7da4699ff922, username=Johann-20.3)

In [2]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk

val ACCOUNTS_YAML_FILE = "../../data/provision/accounts.yaml"
val FUNDS_YAML_FILE = "../../data/provision/funds.yaml"
val INITIAL_BALANCE_YAML_FILE = "../../data/provision/initial-balance.yaml"
val IMPORT_CONFIGURATION_JSON_FILE = "../../data/provision/import-configuration.json"
val IMPORT_CONFIGURATION_YAML_FILE = "../../data/provision/import-configuration.yaml"


In [3]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking
import com.charleskorn.kaml.Yaml
import kotlinx.serialization.builtins.ListSerializer

val accounts = runBlocking { client.provisionAccounts(user, File(ACCOUNTS_YAML_FILE)) }
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=0c2242fa-4f5c-413d-b7e3-ef310f9cadaa, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=800f7794-7f93-45e6-a2e7-62ec846193f1, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=3ca59ece-776f-4449-81f3-05ada06c6df9, name=ING RON, unit=Currency(value=RON))
AccountTO(id=5b168ae1-0013-459e-8f5c-b704394198b6, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=7cdb1130-b163-4084-a459-4fffaa48167c, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=5d8b4623-8fdd-4c30-8560-8dbf97bba713, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=19a667ae-2e08-480c-ac95-ac61a1faba99, name=Other, unit=Currency(value=RON))
AccountTO(id=0e0c4b38-2836-4e08-868a-f269402760dc, name=BT RON, unit=Currency(value=RON))
AccountTO(id=d96a9851-d246-494d-aac1-4e4f5ba92826, name=BT Economy RON, unit=Currency(value=RON))

In [4]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking { client.provisionFunds(user, File(FUNDS_YAML_FILE)) }
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=b9131a5b-a1a0-4cf6-bf38-0116f54dffe1, name=Expenses)
FundTO(id=bfc464c6-9b48-451b-9ae3-b95951de216a, name=Work Income)
FundTO(id=56111bdd-a072-4771-afff-5830aa1d2d04, name=Savings)
FundTO(id=0cc7d1b9-e781-4080-971f-e5bd507c7d3b, name=Gifts)

In [5]:
import ro.jf.funds.fund.api.model.*
import java.math.BigDecimal
import kotlin.collections.List
import kotlinx.datetime.LocalTime

val initialBalancesTransactions = runBlocking {
    client.provisionInitialBalances(
        user, accounts, funds, File(INITIAL_BALANCE_YAML_FILE),
    )
}
initialBalancesTransactions

[FundTransactionTO(id=27aef585-9946-4f40-be28-71f13521d0da, userId=e972a763-dd79-40e9-8ba9-7da4699ff922, dateTime=2018-12-31T00:00, records=[FundRecordTO(id=44fc3b60-8d93-46ea-a81b-13a3a8254caf, fundId=b9131a5b-a1a0-4cf6-bf38-0116f54dffe1, accountId=0c2242fa-4f5c-413d-b7e3-ef310f9cadaa, amount=100000.0, unit=Currency(value=RON), labels=[])])]

In [6]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val importTask = runBlocking {
    val csvFiles =
        listOf(File("../../data/wallet/2019"), File("../../data/wallet/2020"))
            .flatMap { it.listFiles()?.toList() ?: error("no files found") }
    client.importTransactions(user, File(IMPORT_CONFIGURATION_YAML_FILE), csvFiles)
}
importTask


ImportTaskTO(taskId=bdc017ae-cd20-4263-adac-fbc14016b4d7, status=COMPLETED, reason=null)